# Semantic Kernel
![Alt text](image-1.png)         ![Alt text](image.png)

![Alt text](image-2.png)

Install all python libraries

Prepare a semantic kernel instance first, loading also the AI service settings defined in the [Setup notebook](00-getting-started.ipynb):

In [ ]:
!python -m pip install semantic-kernel==0.3.10.dev0
!python -m pip install azure-devops
!python -m pip install semantic-kernel==0.3.10.dev0
!python -m pip install azure-search-documents==11.3.0b1 # you need to copy source as latest release also does not have all the models
#!python -m pip install azure-mgmt-search --pre --upgrade

Import Packages required

In [1]:
import os
import sys
import asyncio
from dotenv import dotenv_values
import semantic_kernel as sk
from semantic_kernel import ContextVariables, Kernel # Context to store variables and Kernel to interact with the kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion ,AzureTextEmbedding # AI services
from semantic_kernel.planning.sequential_planner import SequentialPlanner # Planner
from semantic_kernel.planning.basic_planner import BasicPlanner # Planner
import pandas as pd


kernel = sk.Kernel() # Create a kernel instance
kernel1 = sk.Kernel() #create another kernel instance for not having semanitc function in the same kernel 

useAzureOpenAI = True
config = dotenv_values(".env")
COLLECTION_NAME = "carml-index-sample"

AZURE_COGNITIVE_SEARCH_ENDPOINT = config["AZURE_COGNITIVE_SEARCH_ENDPOINT"]
AZURE_COGNITIVE_SEARCH_ADMIN_KEY = config["AZURE_COGNITIVE_SEARCH_ADMIN_KEY"]
AZURE_OPENAI_API_KEY = config["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT = config["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_EM_DEPLOYMENT_NAME = config["AZURE_OPENAI_EM_DEPLOYMENT_NAME"]
AZURE_OPENAI_DEPLOYMENT_NAME = config["AZURE_OPENAI_DEPLOYMENT_NAME"]
# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
    kernel1.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
    kernel1.add_text_embedding_generation_service("ada", AzureTextEmbedding(AZURE_OPENAI_EM_DEPLOYMENT_NAME, endpoint, api_key))
    kernel1.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
    kernel1.import_skill(sk.core_skills.TextMemorySkill())
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))

In [ ]:
deployment, api_key, endpoint

In [2]:
from plugins.AzureMonitor.Log_Analytics import loganalytics
math_plugin = kernel.import_skill(loganalytics(kernel1), skill_name="AzureMonitor")
variables = ContextVariables()
variables["description"] = "test"

Import the **Semantic functions**:Azuredevops


App requests trend
application Failed requests
Application response time trend
Application usage by contry
container inventory
virtual machines
Accounts Failed to Logon
Active service principal risk detections 
Active user risk detections 
All Azure Activity 
All Security Activities
All SiginLogs events
Failed MFA challenge
Failed login Count 
Avg node CPU usage percentage per minute  
Container CPU 
Signin Locations


In [3]:
import plotly.express as px
variables["title"] = "please share all sign-in location?"
result = await kernel.run_async( math_plugin["lquery"], input_vars=variables )
print(variables["Df"])




SigninLogs| summarize Successful=countif(ResultType==0), Failed=countif(ResultType!=0) by Location
   Location  Successful  Failed
0        US         983      17
1        GB         157       7
2        AU         154     111
3        FR         174       1
4        IN         188      16
5        IL         128       0
6        VN          63       8
7        ES          41       0
8        NL          32       3
9        TR          31       2
10       BR          28       0
11       SG          33       0
12       TH          24       0
13       CA          44       0
14       JP          24       0
15       DE          27       0
16       CH          25       0
17       BE          16       0
18       IT           9       2
19       IE         133       2
20       PH          15       2
21       MY          15       0
22       PK          14       0
23       MX          13       3
24       SE           6       3
25       EG           7       0
26       ID           9       5
27   

using KQL query

In [4]:
# note: using skills from the samples folder
plugins_directory = "./plugins"

    # Import the semantic functions
DevFunctions=kernel1.import_semantic_skill_from_directory(plugins_directory, "AzureMonitor")



In [8]:
FDesFunction = DevFunctions["KQLquery"]    
resultFD = FDesFunction("please share all sign-in location?")
print(resultFD)


SigninLogs | summarize count() by Location | project Location


In [6]:
from plugins.AzureMonitor.Log_AnalyticsS import sloganalytics
plugins_directory = "./plugins"

    # Import the semantic functions
DevFunctions=kernel1.import_semantic_skill_from_directory(plugins_directory, "AzureMonitor")

math_plugin = kernel.import_skill(sloganalytics(kernel1), skill_name="AzureMonitor")
variables = ContextVariables()
variables["description"] = "test"

In [9]:
variables["title"] = "please share all sign-in location with successful and failed count?"
result = await kernel.run_async( math_plugin["slquery"], input_vars=variables )
print(variables["Df"])

we are here
SigninLogs | summarize Successful=countif(ResultType==0), Failed=countif(ResultType!=0) by Location
SigninLogs | summarize Successful=countif(ResultType==0), Failed=countif(ResultType!=0) by Location
   Location  Successful  Failed
0        US         982      17
1        GB         157       7
2        AU         154     112
3        FR         174       1
4        IN         188      16
5        IL         128       0
6        VN          63       8
7        ES          41       0
8        NL          32       3
9        TR          31       2
10       BR          28       0
11       SG          33       0
12       TH          24       0
13       CA          44       0
14       JP          24       0
15       DE          27       0
16       CH          25       0
17       BE          16       0
18       IT           9       2
19       IE         133       2
20       PH          15       2
21       MY          15       0
22       PK          14       0
23       MX         

In [ ]:
from azure.monitor.query import LogsQueryClient, LogsQueryStatus
from azure.core.exceptions import HttpResponseError
from azure.identity import DefaultAzureCredential
from datetime import timedelta      
#txtquery = str(resultFD)
credential  = DefaultAzureCredential()
client = LogsQueryClient(credential)
#txtquery = "'"+ txtquery + "'"
#print(txtquery)  
txtquery="SigninLogs | summarize count() by Location | project Location"
#txtquery='"' + txtquery + '"' 
print(txtquery)
LOGS_WORKSPACE_ID="81a662b5-8541-481b-977d-5d956616ac5e"
response = client.query_workspace(LOGS_WORKSPACE_ID,txtquery,timespan=timedelta(days=1))
try: 
            print(txtquery)           
            if response.status == LogsQueryStatus.PARTIAL:
                print(txtquery +" partial")
                error = response.partial_error
                data = response.partial_data
            elif response.status == LogsQueryStatus.SUCCESS:
                print(txtquery+" Success")
                viz = response.visualization
                data = response.tables
            for table in data:
                df = pd.DataFrame(data=table.rows, columns=table.columns)
                context["df"] = df
                #return df
except HttpResponseError as err:
            print (err)
            #return err
       